In [2]:
import pandas as pd
import numpy as np 

In [3]:
data = pd.read_csv("C:/Users/vlith/OneDrive/Documents/Downloads/weatherHistory.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
Formatted Date              96453 non-null object
Summary                     96453 non-null object
Precip Type                 95936 non-null object
Temperature (C)             96453 non-null float64
Apparent Temperature (C)    96453 non-null float64
Humidity                    96453 non-null float64
Wind Speed (km/h)           96453 non-null float64
Wind Bearing (degrees)      96453 non-null float64
Visibility (km)             96453 non-null float64
Loud Cover                  96453 non-null float64
Pressure (millibars)        96453 non-null float64
Daily Summary               96453 non-null object
dtypes: float64(8), object(4)
memory usage: 8.8+ MB


In [4]:
from sklearn.model_selection import train_test_split

def normalize(data):
    data_mean = data.mean(axis=0)
    data_std = data.std(axis=0)
    return (data - data_mean) / data_std

# Split the dataset into features and target
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Split the dataset into training and testing parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

def build_lstm_model(input_shape, num_layers, units_per_layer, dropout_rate):
    model = Sequential()
    
    # Adding the first LSTM layer
    model.add(LSTM(units=units_per_layer[0], return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Adding subsequent LSTM layers
    for i in range(1, num_layers):
        # Return sequences for all layers except the last one
        return_sequences = True if i < num_layers - 1 else False
        model.add(LSTM(units=units_per_layer[i], return_sequences=return_sequences))
        model.add(Dropout(dropout_rate))
    
    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))  # Adjust units and activation function based on the task
    
    return model

# Example usage
input_shape = (data)  # Define your input shape
num_layers = 3  # Number of LSTM layers
units_per_layer = [64, 64, 64]  # Number of units in each LSTM layer
dropout_rate = 0.2  # Dropout rate to prevent overfitting

model = build_lstm_model(input_shape, num_layers, units_per_layer, dropout_rate)
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(loss='mean_squared_error', optimizer=Adam(), metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Plot training and validation loss
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()